In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

df = pd.read_excel('./menu.xlsx',
                   header=0,
                   dtype=str).fillna('')

list_url = df['Url'].tolist()

print('总数量：' + str(len(list_url)))
print()

# = = = = = = = = = = = = = = =

df = pd.DataFrame()

# = = = = = = = = = = = = = = =

import requests

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for url in list_url:
    work.put_nowait(url)

def crawler():
    global df
    
    while not work.empty():
        url = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        status = 'error'
        for _ in range(666):
            try:
                resp = requests.get(url,
                                    headers=UA.get_User_Agent_Requests(),
                                    proxies=Proxy.get_Proxy_Requests()).text
                
                # = = = = = = = = = = = = = = =
                
                soup = BeautifulSoup(resp, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))

                list_make_model_engine = [_.strip() for _ in html.xpath('//h3[@class="part__titre-groupe-vehicules"]/text()')]
                
                # = = = = = = = = = = = = = = =
                
                list_list_year = []
                for table in html.xpath('//table[@class="part__table-applications"]'):
                    list_list_year.append([int(year.strip()) for year in table.xpath('./tbody/tr/td[2]/text()')])
                
                # = = = = = = = = = = = = = = =
                
                list_vehicle = []
                while True:
                    for make_model_engine, list_year in zip(list_make_model_engine, list_list_year):
                        year_max = max(list_year)
                        year_min = min(list_year)
                        if year_max == year_min:
                            list_vehicle.append(make_model_engine + ' ' + str(year_min))
                        else:
                            list_vehicle.append(make_model_engine + ' ' + str(year_min) + '-' + str(year_max))
                            
                    # = = = = = = = = = = = = = = =
                    
                    if len(html.xpath('//li[@class="paginator__next"]/a')) == 0:
                        break
                    else:
                        resp = requests.get('https://ecat.spectrapremium.com' + html.xpath('//li[@class="paginator__next"]/a/@href')[0].strip(),
                                            headers=UA.get_User_Agent_Requests(),
                                            proxies=Proxy.get_Proxy_Requests()).text
                        soup = BeautifulSoup(resp, 'lxml')
                        html = etree.HTML(str(soup))
                        list_make_model_engine = [_.strip() for _ in html.xpath('//h3[@class="part__titre-groupe-vehicules"]/text()')]
                        list_list_year = []
                        for table in html.xpath('//table[@class="part__table-applications"]'):
                            list_list_year.append([int(year.strip()) for year in table.xpath('./tbody/tr/td[2]/text()')])

                # = = = = = = = = = = = = = = =

                vehicle = '\n'.join(list_vehicle)
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame([{'status': 'ok',
                                         'Url': url,
                                         'Vehicle': vehicle}])
                
                status = 'ok'
                
                break
                
            except:
                time.sleep(1)
                continue

        if status == 'error':
            df_temp = pd.DataFrame([{'status': 'error',
                                     'Url': url}])

        df = pd.concat([df, df_temp], ignore_index=True).fillna('')

        print(url +  '  <->  [' + status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

df = df.drop_duplicates(ignore_index=True)

print()
print('爬虫输出中')
df.to_excel('./vehicle_' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
print()
print('爬虫结束')

总数量：561

https://ecat.spectrapremium.com/en/parts/GMP102A  <->  [ok] - 剩余数量：541 - 当前时间：17:13:57
https://ecat.spectrapremium.com/en/parts/FP96A  <->  [ok] - 剩余数量：540 - 当前时间：17:13:57
https://ecat.spectrapremium.com/en/parts/GMP103A  <->  [ok] - 剩余数量：539 - 当前时间：17:13:57
https://ecat.spectrapremium.com/en/parts/GMP112A  <->  [ok] - 剩余数量：538 - 当前时间：17:13:57
https://ecat.spectrapremium.com/en/parts/GMP07B  <->  [ok] - 剩余数量：537 - 当前时间：17:13:57
https://ecat.spectrapremium.com/en/parts/GMP105A  <->  [ok] - 剩余数量：536 - 当前时间：17:13:58
https://ecat.spectrapremium.com/en/parts/GMP11A  <->  [ok] - 剩余数量：535 - 当前时间：17:13:58
https://ecat.spectrapremium.com/en/parts/GMP114A  <->  [ok] - 剩余数量：534 - 当前时间：17:13:58
https://ecat.spectrapremium.com/en/parts/GMP115A  <->  [ok] - 剩余数量：533 - 当前时间：17:13:58
https://ecat.spectrapremium.com/en/parts/GMP101A  <->  [ok] - 剩余数量：532 - 当前时间：17:13:58
https://ecat.spectrapremium.com/en/parts/GMP128A  <->  [ok] - 剩余数量：531 - 当前时间：17:14:00
https://ecat.spectrapremium.com/en/par

https://ecat.spectrapremium.com/en/parts/CRP33A  <->  [ok] - 剩余数量：445 - 当前时间：17:16:05
https://ecat.spectrapremium.com/en/parts/CRP39A  <->  [ok] - 剩余数量：444 - 当前时间：17:16:08
https://ecat.spectrapremium.com/en/parts/CRP40A  <->  [ok] - 剩余数量：443 - 当前时间：17:16:10
https://ecat.spectrapremium.com/en/parts/CRP32A  <->  [ok] - 剩余数量：442 - 当前时间：17:16:13
https://ecat.spectrapremium.com/en/parts/CRP43A  <->  [ok] - 剩余数量：441 - 当前时间：17:16:20
https://ecat.spectrapremium.com/en/parts/CRP35A  <->  [ok] - 剩余数量：440 - 当前时间：17:16:29
https://ecat.spectrapremium.com/en/parts/CRP44A  <->  [ok] - 剩余数量：439 - 当前时间：17:16:30
https://ecat.spectrapremium.com/en/parts/CRP38A  <->  [ok] - 剩余数量：438 - 当前时间：17:16:30
https://ecat.spectrapremium.com/en/parts/CRP44B  <->  [ok] - 剩余数量：437 - 当前时间：17:16:31
https://ecat.spectrapremium.com/en/parts/CRP41A  <->  [ok] - 剩余数量：436 - 当前时间：17:16:33
https://ecat.spectrapremium.com/en/parts/CRP42A  <->  [ok] - 剩余数量：435 - 当前时间：17:16:35
https://ecat.spectrapremium.com/en/parts/CRP46A  <->  

https://ecat.spectrapremium.com/en/parts/BMP13A  <->  [ok] - 剩余数量：349 - 当前时间：17:19:16
https://ecat.spectrapremium.com/en/parts/BMP16A  <->  [ok] - 剩余数量：348 - 当前时间：17:19:19
https://ecat.spectrapremium.com/en/parts/BMP17A  <->  [ok] - 剩余数量：347 - 当前时间：17:19:25
https://ecat.spectrapremium.com/en/parts/DTP04B  <->  [ok] - 剩余数量：346 - 当前时间：17:19:26
https://ecat.spectrapremium.com/en/parts/BMP11A  <->  [ok] - 剩余数量：345 - 当前时间：17:19:27
https://ecat.spectrapremium.com/en/parts/BMP19A  <->  [ok] - 剩余数量：344 - 当前时间：17:19:28
https://ecat.spectrapremium.com/en/parts/GMP26A  <->  [ok] - 剩余数量：343 - 当前时间：17:19:29
https://ecat.spectrapremium.com/en/parts/BMP06A  <->  [ok] - 剩余数量：342 - 当前时间：17:19:29
https://ecat.spectrapremium.com/en/parts/BMP24A  <->  [ok] - 剩余数量：341 - 当前时间：17:19:30
https://ecat.spectrapremium.com/en/parts/BMP21A  <->  [ok] - 剩余数量：340 - 当前时间：17:19:30
https://ecat.spectrapremium.com/en/parts/BMP20A  <->  [ok] - 剩余数量：339 - 当前时间：17:19:32
https://ecat.spectrapremium.com/en/parts/BMP23A  <->  

https://ecat.spectrapremium.com/en/parts/MIP13A  <->  [ok] - 剩余数量：253 - 当前时间：17:21:41
https://ecat.spectrapremium.com/en/parts/CMP02A  <->  [ok] - 剩余数量：252 - 当前时间：17:21:41
https://ecat.spectrapremium.com/en/parts/HYP01B  <->  [ok] - 剩余数量：251 - 当前时间：17:21:41
https://ecat.spectrapremium.com/en/parts/MZP03B  <->  [ok] - 剩余数量：250 - 当前时间：17:21:42
https://ecat.spectrapremium.com/en/parts/MZP03A  <->  [ok] - 剩余数量：249 - 当前时间：17:21:42
https://ecat.spectrapremium.com/en/parts/CRP19A  <->  [ok] - 剩余数量：248 - 当前时间：17:21:43
https://ecat.spectrapremium.com/en/parts/MZP02A  <->  [ok] - 剩余数量：247 - 当前时间：17:21:43
https://ecat.spectrapremium.com/en/parts/MZP06A  <->  [ok] - 剩余数量：246 - 当前时间：17:21:43
https://ecat.spectrapremium.com/en/parts/MZP07A  <->  [ok] - 剩余数量：245 - 当前时间：17:21:43
https://ecat.spectrapremium.com/en/parts/CMP04A  <->  [ok] - 剩余数量：244 - 当前时间：17:21:44
https://ecat.spectrapremium.com/en/parts/MZP05A  <->  [ok] - 剩余数量：243 - 当前时间：17:21:44
https://ecat.spectrapremium.com/en/parts/MZP15A  <->  

https://ecat.spectrapremium.com/en/parts/MIP03A  <->  [ok] - 剩余数量：157 - 当前时间：17:22:38
https://ecat.spectrapremium.com/en/parts/NSP40A  <->  [ok] - 剩余数量：156 - 当前时间：17:22:38
https://ecat.spectrapremium.com/en/parts/MDP12A  <->  [ok] - 剩余数量：155 - 当前时间：17:22:39
https://ecat.spectrapremium.com/en/parts/NSP39A  <->  [ok] - 剩余数量：154 - 当前时间：17:22:39
https://ecat.spectrapremium.com/en/parts/NSP45A  <->  [ok] - 剩余数量：153 - 当前时间：17:22:39
https://ecat.spectrapremium.com/en/parts/NSP38A  <->  [ok] - 剩余数量：152 - 当前时间：17:22:40
https://ecat.spectrapremium.com/en/parts/NSP44A  <->  [ok] - 剩余数量：151 - 当前时间：17:22:40
https://ecat.spectrapremium.com/en/parts/NSP37A  <->  [ok] - 剩余数量：150 - 当前时间：17:22:40
https://ecat.spectrapremium.com/en/parts/NSP47A  <->  [ok] - 剩余数量：149 - 当前时间：17:22:41
https://ecat.spectrapremium.com/en/parts/SAP01A  <->  [ok] - 剩余数量：148 - 当前时间：17:22:41
https://ecat.spectrapremium.com/en/parts/NSP46A  <->  [ok] - 剩余数量：147 - 当前时间：17:22:41
https://ecat.spectrapremium.com/en/parts/SUP04B  <->  

https://ecat.spectrapremium.com/en/parts/GMP83A  <->  [ok] - 剩余数量：61 - 当前时间：17:24:55
https://ecat.spectrapremium.com/en/parts/HOP01A  <->  [ok] - 剩余数量：60 - 当前时间：17:24:56
https://ecat.spectrapremium.com/en/parts/GMP55A  <->  [ok] - 剩余数量：59 - 当前时间：17:24:56
https://ecat.spectrapremium.com/en/parts/VWP45A  <->  [ok] - 剩余数量：58 - 当前时间：17:24:57
https://ecat.spectrapremium.com/en/parts/GMP89A  <->  [ok] - 剩余数量：57 - 当前时间：17:24:58
https://ecat.spectrapremium.com/en/parts/VWP47A  <->  [ok] - 剩余数量：56 - 当前时间：17:24:59
https://ecat.spectrapremium.com/en/parts/VWP44A  <->  [ok] - 剩余数量：55 - 当前时间：17:25:01
https://ecat.spectrapremium.com/en/parts/HYP23A  <->  [ok] - 剩余数量：54 - 当前时间：17:25:03
https://ecat.spectrapremium.com/en/parts/GMP75A  <->  [ok] - 剩余数量：53 - 当前时间：17:25:03
https://ecat.spectrapremium.com/en/parts/VWP58A  <->  [ok] - 剩余数量：52 - 当前时间：17:25:04
https://ecat.spectrapremium.com/en/parts/VWP57A  <->  [ok] - 剩余数量：51 - 当前时间：17:25:08
https://ecat.spectrapremium.com/en/parts/GMP98A  <->  [ok] - 剩余数量